# Compare signals

In [ ]:
# general use packages
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np

import regionmask
import geopandas as gp
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
def detrend(dat, dim, deg):
    """ linear detrend dat along the axis dim """
    params = dat.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(dat[dim], params.polyfit_coefficients)
    dat = dat-fit
    return dat, fit, params

In [ ]:
var = "TEMP"
deg = 2
level = 0.9

In [ ]:
# load in FOSI
ds = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/' + var + '.monthly.surface.regrid.nc')[var]
ds['time'] = pd.date_range("1958-01", "2020-12", freq="MS")

## Seasonal climatology (amplitude? or standard deviation?)

In [ ]:
# calculate amplitude of seasonal cycle
max = ds.groupby('time.month').mean().max('month')
min = ds.groupby('time.month').mean().min('month')

In [ ]:
seas_climo = ds.groupby('time.month').mean()
seas_climo_std = ds.groupby('time.month').std()

In [ ]:
seas_amp = max - min
seas_std = seas_climo.std('month')

In [ ]:
# deseason
ds_deseas = ds.groupby('time.month') - ds.groupby('time.month').mean()

In [ ]:
seas_amp.plot(vmin = 0, vmax=3)
plt.show()
seas_std.plot(vmin = 0, vmax=3)

## trend (per decade)

In [ ]:
# Calculate trend
ds_detr, fit, params = detrend(ds_deseas, 'time', deg)

trend = fit[-1] - fit[0]

In [ ]:
(trend / 63 * 10).plot()

## interannual variability (standard deviation)

In [ ]:
# interannual variability
interannual_var =  ds_detr.groupby('time.year').mean().std('year')

interannual_var.plot()

## Extremes

In [ ]:
# # magnitude of extremes
# fosi_extremes = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/' + var + '.fosi.rolling.thold.Rolling.nc')['threshold']
# fosi_extremes

threshold = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/' + var + '.fosi.rolling.thold.Rolling.' + str(level) + '.reviewer.nc')['__xarray_dataarray_variable__']

In [ ]:
threshold_var = (threshold.mean('month') - ds_detr.groupby('time.month').mean().mean('month'))#.plot(vmin=0,vmax=2,extend='max')

## Figure

In [ ]:
# ## plot omega_arag
# # trend, interannual standard deviation, amplitude of seasonal cycle, extremes magnitude
# f, ax = plt.subplots(4,1,subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)),figsize=(6,10))

# (trend/63 * 10).plot(ax=ax[0],transform = ccrs.PlateCarree(), cmap = 'BuPu_r', vmin = -0.2, vmax = 0.0,extend='min')
# ax[0].add_feature(cfeature.LAND, color='k', zorder=3)
# ax[0].set_title('Trend (per decade)')

# (interannual_var).plot(ax=ax[2],transform = ccrs.PlateCarree(),cmap = 'BuPu',vmin=-0,vmax=0.2,extend='max')
# ax[2].add_feature(cfeature.LAND, color='k', zorder=3)
# ax[2].set_title('Stand. Dev. of Interannual Cycle')

# (seas_std).plot(ax=ax[1],transform = ccrs.PlateCarree(),cmap = 'BuPu',vmin=-0,vmax=0.2,extend='max')
# ax[1].add_feature(cfeature.LAND, color='k', zorder=3)
# ax[1].set_title('Stand. Dev. of Seasonal Cycle')

# (threshold_var).plot(ax=ax[3],transform = ccrs.PlateCarree(),cmap = 'BuPu_r',vmin=-0.2,vmax=0,extend='min')
# ax[3].add_feature(cfeature.LAND, color='k', zorder=3)
# ax[3].set_title('Average Threshold Anomaly')

# f.savefig(var + '.compare_magnitude.pdf')

In [ ]:
## plot temperature
# trend, interannual standard deviation, amplitude of seasonal cycle, extremes magnitude
f, ax = plt.subplots(4,1,subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)),figsize=(6,10))

(trend/63 * 10).plot(ax=ax[0],transform = ccrs.PlateCarree(), cmap = 'BuPu', vmin = -0, vmax = 3,extend='max')
ax[0].add_feature(cfeature.LAND, color='k', zorder=3)
ax[0].set_title('Trend (per decade)')


(seas_std).plot(ax=ax[1],transform = ccrs.PlateCarree(),cmap = 'BuPu',vmin=-0,vmax=3,extend='max')
ax[1].add_feature(cfeature.LAND, color='k', zorder=3)
ax[1].set_title('Stand. Dev. of Seasonal Cycle')

(interannual_var).plot(ax=ax[2],transform = ccrs.PlateCarree(),cmap = 'BuPu',vmin=-0,vmax=3,extend='max')
ax[2].add_feature(cfeature.LAND, color='k', zorder=3)
ax[2].set_title('Stand. Dev. of Interannual Cycle')

(threshold_var).plot(ax=ax[3],transform = ccrs.PlateCarree(),cmap = 'BuPu',vmin=0,vmax=3,extend='max')
ax[3].add_feature(cfeature.LAND, color='k', zorder=3)
ax[3].set_title('Average Threshold Anomaly')

f.savefig(var + '.compare_magnitude.pdf')